# Evolving surfaces

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
import time
from scipy.interpolate import interp1d
import theano
import theano.tensor as tt

In [ ]:
starry.config.lazy = True
starry.config.quiet = True

## Generate

In [ ]:
np.random.seed(0)

Parameters:

In [ ]:
ydeg = 20
ypad = 5
inc = 75.0
prot = 1.0
alpha = 0.01

nspots = 30
tau_mu = 10 * prot
tau_sig = np.sqrt(tau_mu)
dtau = 2.0
tpad = 50.0

tmax = 100
npts = 1001
t = np.linspace(0, tmax, npts)
dt = t[1] - t[0]

Generate the spot expansions:

In [ ]:
map = starry.Map(ydeg)

# Spot latitude distribution: isotropic
lat = lambda: (np.arccos(2 * np.random.random() - 1) - 0.5 * np.pi) * 180 / np.pi

# Spot longitude distribution: isotropic
lon = lambda: 360.0 * np.random.random()

# Spot size distribution
sigma = lambda: max(0.01, np.exp(-3.5 + 0.4 * np.random.randn()))

# Spot intensity distribution
intensity = lambda: -min(0.5, np.exp(-3 + 0.5 * np.random.randn()))

# Generate the Ylm coeffs for each spot
y = np.empty((nspots, (ydeg + 1) ** 2))
for n in range(nspots):
    map.reset()
    map.add_spot(lat=lat(), lon=lon(), sigma=sigma(), intensity=intensity())
    y[n] = (map.amp * map.y).eval()
y[:, (ydeg - ypad + 1) ** 2 :] = 0
y[:, 0] = 0

In [ ]:
nx = 1 + int(np.ceil(np.sqrt(nspots)))
ny = 1
while ny * nx < nspots:
    ny += 1
fig, ax = plt.subplots(ny, nx, figsize=(12, 5))
ax = ax.flatten()
for axis in ax:
    axis.axis("off")
for k in range(nspots):
    map.reset()
    map[1:, :] = y[k, 1:]
    img = np.pi * map.render(projection="moll", res=100).eval()
    ax[k].imshow(
        img,
        origin="lower",
        extent=(-1, 1, -0.5, 0.5),
        cmap="Greys_r",
        vmin=0.9,
        vmax=1,
    )
    x_el = np.linspace(-1, 1, 1000)
    y_el = 0.5 * np.sqrt(1 - x_el ** 2)
    ax[k].plot(x_el, y_el, "k-", lw=1, clip_on=False)
    ax[k].plot(x_el, -y_el, "k-", lw=1, clip_on=False)

Get the spot timescales:

In [ ]:
tau = tau_mu + tau_sig * np.random.randn(nspots)

In [ ]:
plt.hist(tau)
plt.xlabel("timescale [days]");

Get the spot emergence times and phases:

In [ ]:
t0 = np.sort((tmax + tpad) * np.random.random(nspots) - tpad)
theta = (360.0 / prot) * (t.reshape(1, -1) - t0.reshape(-1, 1))

In [ ]:
plt.hist(t0)
plt.xlabel("emergence time [days]");

Get the spot amplitudes as a function of time:

In [ ]:
a = np.exp(
    -((t.reshape(1, -1) - (t0 + dtau * tau).reshape(-1, 1)) ** 2)
    / (2 * tau.reshape(-1, 1) ** 2)
)

In [ ]:
plt.imshow(a, aspect="auto", extent=(0, tmax, nspots, 0), vmin=0, vmax=1)
plt.colorbar(label="amplitude")
plt.plot(t0, 0.5 + np.arange(nspots), "w|", ms=7.5)
plt.xlim(0, tmax)
plt.xlabel("time [days]")
plt.ylabel("spot number");

Visualize the star:

In [ ]:
def _get_image(_y, _theta):
    map = starry.Map(ydeg, inc=inc, alpha=alpha, tau=np.inf)
    map[1:, :] = _y[1:]
    return np.pi * map.render(projection="moll", res=300, theta=_theta) - 1.0


with theano.configparser.change_flags(compute_test_value="off"):
    _y = tt.dvector()
    _theta = tt.dvector()
    get_image = theano.function([_y, _theta], _get_image(_y, _theta))


downsamp = 10
nim = len(t[::downsamp])
img = np.ones((nim, 300, 300))
for k in tqdm(range(nspots)):
    img += a[k, ::downsamp].reshape(-1, 1, 1) * get_image(y[k], theta[k, ::downsamp])
map.show(image=img, projection="moll", colorbar=True)

Get the light curve:

In [ ]:
def get_model(
    t=t, y=y, a=a, theta=theta, prot=prot, inc=inc, alpha=alpha, tpad=tpad, fast=True
):

    map.inc = inc
    map.alpha = alpha
    map.tau = np.inf
    model = np.ones(npts)
    nspots = len(y)

    if fast:

        theta0 = (
            360.0
            / prot
            * np.concatenate(
                (-t[::-1], t[1:], np.arange(t[-1], t[-1] + tpad, t[1] - t[0])[1:],)
            )
        )
        X0 = map.design_matrix(theta=theta0).eval()
        for k in range(nspots):
            i = np.argmax(theta0 > theta[k, 0]) - 1
            Xa = X0[i : i + npts]
            Xb = X0[i + 1 : i + 1 + npts]
            X = (
                Xa * (theta0[i] - theta[k, 0]) + Xb * (theta0[i + 1] - theta[k, 0])
            ) / (theta0[i + 1] - theta0[i])
            model += a[k] * X0[i : i + npts].dot(y[k])

    else:

        for k in range(nspots):
            X = map.design_matrix(theta=theta[k]).eval()
            model += a[k] * X.dot(y[k])

    return model

In [ ]:
def plot_lc(t, fluxes, styles=None, nrow=5, ncol=3, figsize=(12, 10)):

    fig = plt.figure(figsize=figsize)
    ax_main = plt.subplot2grid((nrow, ncol), (0, 0), colspan=ncol, rowspan=2)
    ax_sub = [
        plt.subplot2grid((nrow, ncol), (2 + i, j))
        for i in range(nrow - 2)
        for j in range(ncol)
    ]
    nsub = len(ax_sub)
    npts = len(t)

    if styles is None:
        styles = [dict() for flux in fluxes]
    for flux, style in zip(fluxes, styles):
        ax_main.plot(t, flux, **style)

        for k, ax in enumerate(ax_sub):

            a = int(k * npts / nsub)
            b = int((k + 1) * npts / nsub)
            ax.plot(t[a:b], flux[a:b], **style)

    ax_main.legend(fontsize=8, loc="lower left")

    for label in ax_main.get_yticklabels() + ax_main.get_xticklabels():
        label.set_fontsize(10)
    for ax in ax_sub:
        for label in ax.get_yticklabels() + ax.get_xticklabels():
            label.set_fontsize(8)
    ax_main.set_ylabel("flux")
    for ax in ax_sub[-ncol:]:
        ax.set_xlabel("time [days]", fontsize=12)
    for ax in ax_sub[::ncol]:
        ax.set_ylabel("flux", fontsize=12)

In [ ]:
flux0 = get_model(alpha=0, fast=False)
flux = get_model(alpha=alpha, fast=False)

In [ ]:
plt.hist(tau);

In [ ]:
plt.hist(tau);

In [ ]:
plt.hist(tau);

In [ ]:
plot_lc(
    t,
    [flux0, flux],
    styles=[
        dict(color="C1", lw=1, alpha=0.5, label="solid"),
        dict(color="C0", lw=2, label="diff rot"),
    ],
)

## Inference

In [ ]:
import pymc3 as pm
import pymc3.distributions.transforms as tr
import exoplanet as exo
import theano.tensor as tt

In [ ]:
alpha_tru = alpha
t0_tru = np.array(t0)
tau_tru = np.array(tau)
y_tru = np.array(y)
nspots_tru = nspots

In [ ]:
ferr = 1e-4
fobs = flux + ferr * np.random.randn(len(flux))

In [ ]:
plot_lc(
    t,
    [flux, fobs],
    styles=[
        dict(color="C0", lw=1, alpha=0.5, label="true"),
        dict(color="k", ls="None", marker=".", ms=2, label="observed"),
    ],
)

In [ ]:
ymu = np.mean(y_tru, axis=0)
ycov = np.cov(y_tru.T)
ycov += 1e-12 * np.eye(len(ymu))
nspots = 10

In [ ]:
map = starry.Map(ydeg)
map[:, :] = np.random.multivariate_normal(ymu, ycov)
map.show(projection="moll")

In [ ]:
# Guess: evenly spaced
t0_guess = np.linspace(-tpad + 0.01, tmax - 0.01, nspots)

# Guess
alpha_guess = 0.1
tau_guess = 1.0
delta_guess = 0.0

# Phases
theta = (360.0 / prot) * (t.reshape(1, -1) - tt.reshape(t0_guess, (-1, 1)))

# Solve for the ylms at each epoch
map = starry.Map(ydeg, inc=inc, alpha=alpha_guess, tau=tau_guess, delta=delta_guess)
y_guess = [None for n in range(nspots)]
npns = int(np.floor(npts / nspots))
for n in tqdm(range(nspots)):
    a = np.argmin(np.abs(t - t0_guess[n]))
    b = a + npns
    start = max(0, a - npns // 2)
    stop = min(npts, a + npns // 2)
    idx = slice(start, stop)
    map.set_data(fobs[idx] - 1.0, C=ferr ** 2)
    map.set_prior(mu=ymu, L=ycov)
    y, _ = map.solve(theta=theta[n][idx])
    y_guess[n] = y.eval()

In [ ]:
with pm.Model() as model:

    # Spot emergence times
    t0 = pm.Uniform(
        "t0",
        -tpad,
        tmax,
        shape=nspots,
        # transform=tr.Ordered(),
        testval=t0_guess,
    )

    # Differential rotation
    alpha = pm.Uniform("alpha", 0.0, 1.0, testval=alpha_guess)
    tau = pm.Uniform("tau", 0.0, 2.0, testval=tau_guess)
    delta = pm.Uniform("delta", -1.0, 1.0, testval=delta_guess)

    # Phases
    theta = (360.0 / prot) * (t.reshape(1, -1) - tt.reshape(t0, (-1, 1)))

    # Flux model
    map = starry.Map(ydeg, inc=inc, alpha=alpha, tau=tau, delta=delta)
    m = tt.ones(npts)
    for k in range(nspots):
        yk = pm.MvNormal(
            "y{}".format(k), ymu, ycov, shape=(len(ymu),), testval=y_guess[k],
        )
        X = map.design_matrix(theta=theta[k])
        m += tt.dot(X, yk)
    pm.Deterministic("m", m)
    m_guess = exo.eval_in_model(m)

    # Likelihood
    pm.Normal("obs", mu=m, sd=ferr, observed=fobs)

In [ ]:
with model:
    map_soln = exo.optimize(options=dict(maxiter=399))

In [ ]:
plot_lc(
    t,
    [fobs, m_guess, map_soln["m"]],
    styles=[
        dict(color="k", ls="None", marker=".", ms=2, label="observed"),
        dict(color="C1", lw=1, alpha=0.5, label="guess"),
        dict(color="C0", lw=1, alpha=1, label="MAP"),
    ],
    nrow=6,
    ncol=3,
    figsize=(12, 12),
)

In [ ]:
plot_lc(
    t,
    [fobs, map_soln["m"]],
    styles=[
        dict(color="k", ls="None", marker=".", ms=2, label="observed"),
        dict(color="C0", lw=1, alpha=1, label="MAP"),
    ],
    nrow=6, ncol=3, figsize=(12, 12)
)i

In [ ]:
map_tmp = starry.Map(20)
map_tmp[1:, :] = map_soln["y5"][1:]
map_tmp.show(projection="moll")

In [ ]:
map_soln["alpha"]

In [ ]:
map_soln["tau"]

In [ ]:
map_soln["delta"]

In [ ]:
with model:
    trace = pm.sample(
        tune=300,
        draws=1000,
        start=map_soln,
        cores=2,
        chains=2,
        step=exo.get_dense_nuts_step(target_accept=0.9),
    )